In [9]:
%load_ext autoreload
import sys
sys.path.append('../src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%autoreload 2
from graph_coloring import *
from env_test import random_run
from scipy.sparse import csr_matrix
from copy import deepcopy

In [38]:
g = nx.generators.random_graphs.erdos_renyi_graph(100, 0.2, seed=754)
# g = nx.Graph()
# g.add_edges_from([
#     (0, 2), (1, 2), (1, 4), (2, 0), (2, 1), (2, 3), (3, 5), (3, 6), (4, 5)
# ])

In [39]:
c = Coloring(g)

In [40]:
sb = SpectralBound(g)
sb.reward_to_go(c)

-4

In [41]:
%%time
heuristic = GreedyColoring(g)
heuristic.reward_to_go()

CPU times: user 25.4 ms, sys: 2.08 ms, total: 27.5 ms
Wall time: 26.4 ms


-11

In [48]:
%%time
ro = RolloutColoring(g, heuristic, depth=2)
ro_coloring = ro.solve()
len(ro_coloring)

CPU times: user 6.39 s, sys: 19.4 ms, total: 6.4 s
Wall time: 6.41 s


10

In [32]:
lp = LPRelaxation(g)

In [37]:
%%time
lp.reward_to_go(c)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])


INFO:gurobipy:Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])


Thread count: 8 physical cores, 16 logical processors, using up to 16 threads


INFO:gurobipy:Thread count: 8 physical cores, 16 logical processors, using up to 16 threads


Optimize a model with 23177 rows, 2301 columns and 50554 nonzeros


...

INFO:gurobipy:Solved in 546 iterations and 0.08 seconds (0.05 work units)


Optimal objective  5.000000000e-01


INFO:gurobipy:Optimal objective  5.000000000e-01


CPU times: user 437 ms, sys: 62.6 ms, total: 499 ms
Wall time: 344 ms


-1

In [ ]:
from math import floor, sqrt, ceil
import numpy as np
from collections import Counter

In [ ]:
degrees = sorted(dict(g.degree()).values())
m = len(g.edges)
n = len(g.nodes)
d = degrees[-1] + 1  # 1 
l = floor((1 + sqrt(8 * m + 1)) / 2) # 2 
M = max(map(lambda i: min(degrees[-i - 1] , i), range(len(degrees)))) # 3
r = max(Counter(degrees).items(), key=lambda x: x[1] if x[0] > d / 2 else 0)[0] 
q = ceil(r / (r+1) * d) # 5

xi = floor((3 + sqrt(9 + 8 * (m - n))) / 2) # 6